In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
### Mape表をCSVとして保存する関数
### 引数はbenchmark:ベンチマーク名, FixClass:固定するベンチマーククラス, FixProcess:固定するプロセス数, DirPath:保存するディレクトリ
def SaveMapeTables(FixClass="B", FixProcess=256, DirPath = "./tmp_GenerateResources/"):
    for benchmark in benchmarks:
        MapeTableFixedClass = return_MapeTableFixedClass(benchmark, FixClass="B")
        MapeTableFixedProcess = return_MapeTableFixedProcess(benchmark, FixProcess=256)
        FixedClassFileName = f"MapeTableFixedClass_{benchmark}{FixClass}.csv"
        FixedProcessFileName = f"MapeTableFixedProcess_{benchmark}{FixProcess}.csv"
    #     print(f"FixedClassFileName={FixedClassFileName}, MapeTableFixedProcess={FixedProcessFileName}")
        if(type(MapeTableFixedClass) is pd.core.frame.DataFrame):
            print(f"FixedClassFileName={FixedClassFileName}")
            MapeTableFixedClass.to_csv(f"{DirPath}{FixedClassFileName}")
        if(type(MapeTableFixedProcess) is pd.core.frame.DataFrame):
            print(f"FixedProcessFileName={FixedProcessFileName}")
            MapeTableFixedProcess.to_csv(f"{DirPath}{FixedProcessFileName}")


SaveMapeTables(FixClass="C", FixProcess=256)

In [ ]:
# ##### 各関数での実行時間を保存するためのCSVの空データを作成するための処理 #####
# Processes = []
# for i in range(1,16):
#     Processes.append(i*i)
# for j in range(9):
#     Processes.append(2**j)
# Processes = list(set(Processes))
# Processes.sort()
# index = classes
# EmptyDF = pd.DataFrame(index=index, columns=Processes)
# for benchmark in benchmarks:
#     EmptyDF.to_csv(f"./csv_files/ExecTime@{benchmark}.csv")

In [ ]:
### クラス：TimeData
### 下記の関数(= return_TimeDataList())のために作成された
### メソッドreturn_AllData()で値のすべてを辞書形式で受けてることができる
class TimeData:
    def __init__(self, benchmark="cg", process="32", BenchmarkClass="C", time=-1):
        self.benchmark = benchmark
        self.process = process
        self.BenchmarkClass = BenchmarkClass
        self.time = time
    def return_AllData(self):
        return {"benchmark":self.benchmark, "process":self.process, "BenchmarkClass":self.BenchmarkClass, "time":self.time}

### 返値に独自クラスTimeDataのリストを返す
### 引数に実行プロセス数を取る
### 返値のリストの要素は引数のリストのプロセス数で実行されたベンチマークの
### ベンチマーク名・実行プロセス数・ベンチマーククラス・実行時間が記録されたTimeDataクラスのインスタンス
def return_TimeDataList(process = 256):
    return_list = []
    with open(f"./toGetProfile/toGetTime/TimeWith{process}.txt") as f:
        line_count = 0
        for line in f:
            line_count += 1
            if(line_count%3 == 1):
                benchmark = line[1:3].lower()
                # print(f"benchmark={benchmark}, len(benchmark)={len(benchmark)}")
                Data = TimeData(benchmark=benchmark, process=process)
            if(line_count%3 == 2):
                BenchmarkClass = line[-2]
                # print(f"BenchmarkClass={BenchmarkClass}, len(BenchmarkClass)={len(BenchmarkClass)}")
                Data.BenchmarkClass = BenchmarkClass
            if(line_count%3 == 0):
                Time = line[-25:]
                Time = Time.strip()
                # print(f"Time={Time}")
                Data.time = Time
                return_list.append(Data)
    return(return_list)


In [ ]:

### 関数:FillCSV()
### 引数は,benchmark:ベンチマーク名, process:実行プロセス数, BenchmarkClass:ベンチマーククラス, time:実行時間
### 引数として渡された値を適切なCSVに保存する
def FillCSV(benchmark = "cg",process = 256,BenchmarkClass = "A",time = "0.04"):
    time=float(time)
    CSVFilename = f"./csv_files/ExecTime@{benchmark}.csv"
    DataFrame = pd.read_csv(CSVFilename, index_col=0)
    DataFrame.at[BenchmarkClass, f"{process}"] = time
    DataFrame.to_csv(CSVFilename)


time=12345
CSVFilename = f"./csv_files/ExecTime@cg.csv"
DataFrame = pd.read_csv(CSVFilename, index_col=0)
print(DataFrame.columns.tolist())
DataFrame.at[BenchmarkClass, f"{process}"] = time
print(DataFrame)
DataFrame.to_csv(CSVFilename)

time=54321
CSVFilename = f"./csv_files/ExecTime@cg.csv"
DataFrame = pd.read_csv(CSVFilename, index_col=0)
print(DataFrame.columns.tolist())
DataFrame.at[BenchmarkClass, f"{process}"] = time
print(DataFrame)
DataFrame.to_csv(CSVFilename)

DataFrame = pd.read_csv(CSVFilename, index_col=0)
print(DataFrame.columns.tolist())

In [ ]:
# Benchmarks = []
# for benchmark in benchmarks:
#     Benchmarks.append(benchmark.upper())
# print(Benchmarks)

Processes = [8, 16, 32, 64, 128, 256]
for process in Processes:
    TimeDataList = return_TimeDataList(process)
    for DictData in TimeDataList:
        benchmark = DictData.benchmark
        process = DictData.process
        BenchmarkClass = DictData.BenchmarkClass
        time = DictData.time
        FillCSV(benchmark=benchmark, process=process, BenchmarkClass=BenchmarkClass, time=time)

# print(TimeDataList[0].return_AllData())


In [1]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

### 誤差率の表を作成する関数
### 引数は,benchmark:関数名, predict_class:予測したいクラス, predict_process:予測したいプロセス数, FixProcess:固定する実行プロセス数, FixClass:固定するベンチマーククラス
### 返り値はなし
def GenerateErrorRateTable(benchmark = "cg",predict_class = "D",predict_process = 256,FixProcess = 64,FixClass = "B"):
    try:
        ErrorRateFixedProcessDF = return_ErrorRateFixedProcessDF(benchmark=benchmark, FixProcess=FixProcess, predict_class=predict_class)
    except:
        print("実行プロセスを固定した際の誤差率の表を取得するのに失敗したので、CSVとして保存できませんでした。")
    ErrorRateFixedProcessDF.to_csv(f"./tmp_GenerateResources/ErrorRateTableFixedProcess_{benchmark}{FixProcess}.csv")
    try:
        ErrorRateFixedClassDF = return_ErrorRateFixedClassDF(benchmark=benchmark, FixClass=FixClass, predict_process=predict_process)
    except:
        print("ベンチマーククラスを固定した際の誤差率の表を取得するのに失敗したので、CSVとして保存できませんでした。")
    ErrorRateFixedClassDF.to_csv(f"./tmp_GenerateResources/ErrorRateTableFixedClass_{benchmark}{FixClass}.csv")

### 誤差率のデータフレームを返す関数
### return_ErrorRateFixedProcessDF(), return_ErrorRateFixedClassDF()
def return_ErrorRateFixedProcessDF(benchmark="cg", FixProcess=64, predict_class="D"):
    FixProcessDF = return_FixedProcessModelDF(benchmark=benchmark, FixProcess=256)
    # print(FixProcessDF)
    for index in FixProcessDF.index.tolist():
        for column in FixProcessDF.columns.tolist():
            PredictNum = return_Predicted(FixProcessDF.at[index, column], ConvertBencharkClass_inNPB(predict_class))
            FixProcessDF.at[index, column] = int(return_ErrorRate(FunctionName=index, PredictNum=PredictNum, BenchmarkName=benchmark, BenchmarkClass=predict_class, Process=FixProcess)*1000)/1000
    return (FixProcessDF)
def return_ErrorRateFixedClassDF(benchmark="cg", FixClass="B", predict_process=256):
    FixClassDF = return_FixedClassModelDF(benchmark, FixClass=FixClass)
    # print(FixClassDF)
    for index in FixClassDF.index.tolist():
        for column in FixClassDF.columns.tolist():
            PredictNum = return_Predicted(FixClassDF.at[index, column], predict_process)
            FixClassDF.at[index, column] = int(return_ErrorRate(FunctionName=index, PredictNum=PredictNum, BenchmarkName=benchmark, BenchmarkClass=FixClass, Process=predict_process)*1000)/1000
    return(FixClassDF)

for benchmark in benchmarks:
    GenerateErrorRateTable(benchmark=benchmark)

In [4]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb


### 関数：return_ErrorRateFixed<Class or Process>DFwithAverage()
### 引数：benchmark:ベンチマーク名, FixProcess:固定するプロセス数,  predict_class:予測するクラス
### 引数：benchmark:ベンチマーク名, FixClass:固定するクラス数,  predict_process:予測するプロセス数
### 返り値はエラー率の平均が付与されたエラー率の表
def return_ErrorRateFixedProcessDFwithAverate(benchmark="cg", FixProcess=64, predict_class="D"):
    ErrorRateFixedProcessDF = return_ErrorRateFixedProcessDF(benchmark=benchmark, FixProcess=FixProcess, predict_class=predict_class)
    average = ErrorRateFixedProcessDF.mean()
    average.name = "Average"
    return(ErrorRateFixedProcessDF.append(average))
def return_ErrorRateFixedClassDFwithAverage(benchmark="cg", FixClass="B", predict_process=256):
    ErrorRateFixedClsssDF = return_ErrorRateFixedClassDF(benchmark=benchmark, FixClass=FixClass, predict_process=predict_process)
    average = ErrorRateFixedClsssDF.mean()
    average.name = "Average"
    return(ErrorRateFixedClsssDF.append(average))

### 関数：convert_StrListToIntList()
### 引数：要素はすべて整数なリストのプリント出力
### 返り値：数値のリスト
def convert_StrListToIntList(InputList:list):
    InputList = InputList[1:-1]
    ReturnList = [int(x.strip()) for x in InputList.split(',')]
    return(ReturnList)

### 関数：convert_StrListToStrList()
### 引数：要素はすべて一文字のアルファベットなリストのプリント出力
### 返り値：アルファベットのリスト
def convert_StrListToStrList(InputList:list):
    InputList = InputList[1:-1]
    InputList = InputList.replace('\'', '')
    ReturnList = [x.strip() for x in InputList.split(',')]
    return(ReturnList)

### 関数：return_ExecTime()
### 引数：ベンチマーク名, ベンチマーククラス, 実行プロセス数
### 返り値：引数に該当するベンチマークの実行時間
def return_ExecTime(benchmark="cg", BenchmarkClass="B", Process=256):
    FileDir = "./csv_files/"
    FileName = f"ExecTime@{benchmark}.csv"
    ExecTimeDF = pd.read_csv(f"{FileDir}{FileName}", index_col=0)
    TargetNum = ExecTimeDF.at[BenchmarkClass, f"{Process}"]
    if(np.isnan(TargetNum)):
        TargetNum=-1
    return(TargetNum)

### 関数:return_FixClassCost()
### 引数:benchmark=ベンチマーク名, ProcessList=実行プロセスのリスト, BenchmarkClass=ベンチマーククラス
### 返り値：引数の条件に当てはまるコスト
def return_FixClassCost(benchmark="cg", ProcessList = [1, 2, 4], BenchmarkClass ="B"):
    cost = 0
    for process in ProcessList:
        ExecTime = return_ExecTime(benchmark=benchmark, BenchmarkClass=BenchmarkClass, Process=process)
        if(ExecTime < 0):
            return(-1)
        cost += ExecTime * process
    return(cost)

### 関数:return_FixProcessCost()
### 引数:benchmark=ベンチマーク名, BenchmarkClassList=ベンチマーククラスのリスト, Process=実行プロセス
### 返り値：引数の条件に当てはまるコスト
def return_FixProcessCost(benchmark="cg", BenchmarkClassList = ["A", "B", "C"], Process=256):
    cost = 0
    for BenchmarkClass in BenchmarkClassList:
        ExecTime = return_ExecTime(benchmark=benchmark, BenchmarkClass=BenchmarkClass, Process=Process)
        if(ExecTime < 0):
            return(-1)
        cost += ExecTime * Process
    return(cost)

def return_ErrorRateFixedClassAverageAndCosts(benchmark="cg", FixClass="B", predict_process=256):
    ErrorRateFixedClassDFwithAverage = return_ErrorRateFixedClassDFwithAverage( benchmark=benchmark, FixClass=FixClass, predict_process=predict_process)
    ErrorRateFixedClassAverageSeries = ErrorRateFixedClassDFwithAverage.loc["Average"]
    ErrorRateFixedClassAverageDF = pd.DataFrame(ErrorRateFixedClassAverageSeries)
    index = ErrorRateFixedClassAverageDF.index.tolist()
    columns = ErrorRateFixedClassAverageDF.columns.tolist()
    listedIndex = []
    for i in index:
        listedIndex.append(convert_StrListToIntList(i))
    Costs = []
    for i in listedIndex:
        Costs.append(return_FixClassCost(benchmark=benchmark, ProcessList=i, BenchmarkClass=FixClass))
    ErrorRateFixedClassAverageDF["PredictCosts"] = Costs
    return(ErrorRateFixedClassAverageDF)


def return_ErrorRateFixedProcessAverageAndCosts(benchmark="cg", FixProcess=256, predict_class="D"):
    ErrorRateFixedProcessDFwithAverage = return_ErrorRateFixedProcessDFwithAverate(benchmark=benchmark, FixProcess=FixProcess, predict_class=predict_class)
    ErrorRateFixedProcessAverageSeries = ErrorRateFixedProcessDFwithAverage.loc["Average"]
    ErrorRateFixedProcessAverageDF = pd.DataFrame(ErrorRateFixedProcessAverageSeries)
    index = ErrorRateFixedProcessAverageDF.index.tolist()
    columns = ErrorRateFixedProcessAverageDF.columns.tolist()
    listedIndex = []
    for i in index:
        listedIndex.append(convert_StrListToStrList(i))
    Costs = []
    for i in listedIndex:
        Costs.append(return_FixProcessCost(benchmark=benchmark, BenchmarkClassList=i, Process=FixProcess))
    ErrorRateFixedProcessAverageDF["PredictCosts"] = Costs
    return(ErrorRateFixedProcessAverageDF)

benchmark="cg"
FixProcess=64
predict_class="D"
FixClass="B"
predict_process=256
BenchmarkClass="B"
Process=64

def return_ErrorRateFixedClass_AveragePredictCostRealCost(benchmark="cg", FixClass="B", predict_process=256):
    ErrorRateFixedClass = return_ErrorRateFixedClassAverageAndCosts(benchmark=benchmark, FixClass=FixClass, predict_process=predict_process)
    index = ErrorRateFixedClass.index.tolist()
    columns = ErrorRateFixedClass.columns.tolist()
    RealClass=FixClass
    RealProcess = predict_process
    RealTime = return_ExecTime(benchmark=benchmark, BenchmarkClass=RealClass, Process=RealProcess)
    RealCost = RealProcess * RealTime
    RealCostList = [RealCost] * len(index)
    ErrorRateFixedClass["RealCost"] = RealCostList
    return(ErrorRateFixedClass)


def return_ErrorRateFixedProcess_AveragePredictCostRealCost(benchmark="cg", FixProcess=256, predict_class="D"):
    ErrorRateFixedProcess = return_ErrorRateFixedProcessAverageAndCosts(benchmark=benchmark, FixProcess=FixProcess, predict_class=predict_class)
    index = ErrorRateFixedProcess.index.tolist()
    columns = ErrorRateFixedProcess.columns.tolist()
    RealClass= predict_class
    RealProcess = FixProcess
    RealTime = return_ExecTime(benchmark=benchmark, BenchmarkClass=RealClass, Process=RealProcess)
    RealCost = RealProcess * RealTime
    RealCostList = [RealCost] * len(index)
    ErrorRateFixedProcess["RealCost"] = RealCostList
    return(ErrorRateFixedProcess)


for benchmark in benchmarks:
    print(benchmark)
    print(return_ErrorRateFixedClass_AveragePredictCostRealCost(benchmark=benchmark))
    print(return_ErrorRateFixedProcess_AveragePredictCostRealCost(benchmark=benchmark))

bt
                      Average  PredictCosts  RealCost
[1, 4, 16, 64, 256]    0.0004       1636.04     448.0
[1, 4, 16, 64]       0.001025       1188.04     448.0
[1, 4, 16]            0.00145        837.32     448.0
[1, 4]                402.393        525.64     448.0
[1]                   12.3006        247.04     448.0
                          Average  PredictCosts  RealCost
['A', 'B', 'C', 'D']  8.06452e-05      34457.60  32107.52
['A', 'B', 'C']          0.116032       2350.08  32107.52
['A']                    0.131548        138.24  32107.52
cg
                                      Average  PredictCosts  RealCost
[1, 2, 4, 8, 16, 32, 64, 128, 256]  0.0568636        913.82     166.4
[1, 2, 4, 8, 16, 32, 64, 128]        0.102409        747.42     166.4
[1, 2, 4, 8, 16, 32]                 0.283636        537.50     166.4
[1, 2, 4, 8]                           5.4315        372.06     166.4
[1, 2]                                6.71836        210.46     166.4
                  